### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini_F(self, l_c, r_c):
        return (-(l_c / (l_c + r_c)) ** 2
                - (r_c / (l_c + r_c)) ** 2 + 1)

    def __entropy_F(self, l_c, r_c):
        return (-(l_c / (l_c + r_c)) * np.log2(l_c / (l_c + r_c))
                - (r_c / (l_c + r_c)) * np.log2(r_c / (l_c + r_c)))

    def __misclass_F(self, l_c, r_c):
        return 1 - np.maximum(l_c / (l_c + r_c), r_c / (l_c + r_c))

    def __gini(self, l_c, l_s, r_c, r_s):
        return (self.__gini_F(l_c, r_c)
                - l_c / (l_c + r_c) * self.__gini_F(l_s, -l_s + l_c)
                - r_c / (l_c + r_c) * self.__gini_F(r_s, -r_s + r_c))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return (self.__entropy_F(l_c, r_c)
                - l_c / (l_c + r_c) * self.__entropy_F(l_s, l_c - l_s)
                - r_c / (l_c + r_c) * self.__entropy_F(r_s, r_c - r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (self.__misclass_F(l_c, r_c)
                - l_c / (l_c + r_c) * self.__misclass_F(l_s, l_c - l_s)
                - r_c / (l_c + r_c) * self.__misclass_F(r_s, r_c - r_s))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_features)):]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_features)):]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = np.array(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort(axis=1)
        return np.sort(x, axis=1), y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        sort_out = y[0]
        x = x.T
        s_x, s_y = self.__sort_samples(x, y)
        s_y = (s_y == sort_out).astype(int)
        b = np.bincount(s_y[0])
        l_sums = np.cumsum(s_y, axis=1)
        r_sums = b[1] - l_sums
        length = s_y.shape[1]
        l = np.array(range(length))
        l_c = np.array(l + np.zeros(s_y.shape[0]).reshape(-1, 1))
        l_c[:, 0] += 1
        r_c = -l_c + length
        g_matr = self.G_function(l_c, l_sums, r_c, r_sums)
        ind = np.unravel_index(np.argmax(g_matr, axis=None), g_matr.shape)
        try:
            return ((s_x[ind[0], ind[1]] + s_x[ind[0], ind[1] + 1]) / 2.0, 
                    ind[0], g_matr[ind[0], ind[1]])
        except IndexError:
            return s_x[ind[0], ind[1]], ind[0], g_matr[ind[0], ind[1]]

    def __fit_node(self, x, y, node_id, depth):
        y1, c = np.unique(y, return_counts=True)
        y1.sort()
        prob = c / y.size
        y1 = y1[(-c).argsort()]
        if (depth == self.max_depth) or (y.size < self.min_samples_split):
            self.tree[node_id] = (self.__class__.LEAF_TYPE, y1[0], prob)
        elif np.any(prob >= self.sufficient_share):
            self.tree[node_id] = (self.__class__.LEAF_TYPE, y1[0], prob)
        else:
            threshold, feature_id, crit = self.__find_threshold(x, y)
            self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature_id, threshold)
            xl, xr, yl, yr = self.__div_samples(x, y, feature_id, threshold)
            if yl.size > 0 and yr.size > 0:
                self.feature_importances_[feature_id] += y.size * crit / self.fit_count
                self.__fit_node(xl, yl, 2 * node_id + 1, depth + 1)
                self.__fit_node(xr, yr, 2 * node_id + 2, depth + 1)
            else:
                self.tree[node_id] = (self.__class__.LEAF_TYPE, y1[0], prob)

    def fit(self, x, y):
        self.features_ids = self.get_feature_ids(x.shape[1])
        self.num_class = np.unique(y).size
        self.fit_count = x.shape[0]
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 1.86 ms, sys: 845 µs, total: 2.7 ms
Wall time: 1.79 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 6.18 ms, sys: 1.21 ms, total: 7.39 ms
Wall time: 6.29 ms


## Проверка качества работы на wine

In [9]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.890993265993266

In [10]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [25]:
df = pd.read_csv('./data/speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['from'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['wave'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df['mn_sat'] = df['mn_sat'].fillna(value=0.0)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'zipcode'] = df.loc[:, 'zipcode'].str.replace(',', '').astype(np.float)
df['tuition'] = df['tuition'].fillna(value=0.0)
df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df['income'] = df['income'].fillna(value=0.0)
df = df.dropna(subset=['date'])
df = df.drop(['career'], axis=1)
df['career_c'] = df['career_c'].fillna(value=0.0)
df = df.drop(['sports','tvsports','exercise','dining',
              'museums','art','hiking','gaming',
              'clubbing','reading','tv','theater','movies',
              'concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 
                                        'intel1_1', 'fun1_1', 
                                        'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1',
            'intel1_1', 'fun1_1', 
            'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1',
           'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1',
            'intel2_1', 'fun2_1',
            'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
df = df.dropna()
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df = df_male.join(df_female)
df = df.fillna(value=0)

X_df = df.drop(['match'], axis=1)
y_df = df['match']
X = np.array(X_df)
y = np.array(y_df)
X, y

(array([[ 2.45e+02,  2.34e+02,  5.00e-02, ...,  0.00e+00,  0.00e+00,
          0.00e+00],
        [ 2.45e+02,  2.35e+02,  4.30e-01, ...,  0.00e+00,  0.00e+00,
          0.00e+00],
        [ 2.45e+02,  2.36e+02,  1.90e-01, ...,  0.00e+00,  0.00e+00,
          0.00e+00],
        ...,
        [ 5.52e+02,  5.28e+02, -4.60e-01, ...,  0.00e+00,  0.00e+00,
          0.00e+00],
        [ 5.52e+02,  5.29e+02,  6.20e-01, ...,  0.00e+00,  0.00e+00,
          0.00e+00],
        [ 5.52e+02,  5.30e+02,  1.00e-02, ...,  0.00e+00,  0.00e+00,
          0.00e+00]]), array([0, 0, 0, ..., 0, 0, 0]))

## Проверка скорости работы на Speed Dating Data 

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
%time clf.fit(X_train, y_train)

CPU times: user 11.1 ms, sys: 1.25 ms, total: 12.4 ms
Wall time: 11 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [28]:
%time my_clf.fit(X_train, y_train)

CPU times: user 86.3 ms, sys: 13 ms, total: 99.4 ms
Wall time: 98 ms


## Проверка качества работы на Speed Dating Data

In [29]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.4995921310588818

In [30]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5141969831410825

## Задание 4

In [31]:
clf.feature_importances_

array([0.0013816 , 0.33040805, 0.3477187 , 0.05593367, 0.00483378,
       0.00407759, 0.00775412, 0.00980836, 0.        , 0.01142716,
       0.00730336, 0.01218268, 0.00660911, 0.0048834 , 0.00439506,
       0.00657656, 0.00390672, 0.00511657, 0.02249465, 0.0174808 ,
       0.01524706, 0.01510126, 0.01519646, 0.008139  , 0.01198856,
       0.00455784, 0.0107154 , 0.00044183, 0.01303781, 0.00591157,
       0.02199083, 0.        , 0.        , 0.00192295, 0.01145748,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [32]:
my_clf.feature_importances_

array([0.12724651, 0.16227639, 0.17767719, 0.03353002, 0.02347433,
       0.02335139, 0.03207626, 0.08717406, 0.02614901, 0.08463696,
       0.        , 0.00593734, 0.        , 0.03308306, 0.01001001,
       0.31192396, 0.02904769, 0.03608788, 0.00939212, 0.02523369,
       0.01189056, 0.        , 0.        , 0.16150218, 0.10682782,
       0.02845743, 0.05282765, 0.        , 0.0288417 , 0.        ,
       0.05330645, 0.10024224, 0.05943635, 0.        , 0.0299108 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [33]:
new = np.argsort(clf.feature_importances_)[::-1]
new = new[:10]
top10_clf = X_df.columns[new]
top10_clf

Index(['int_corr', 'pid', 'samerace', 'attr1_1', 'attr3_1', 'sinc1_1',
       'intel1_1', 'amb1_1', 'fun1_1', 'amb2_1'],
      dtype='object')

In [34]:
new = np.argsort(my_clf.feature_importances_)[::-1]
new = new[:10]
top10_my_clf = X_df.columns[new]
top10_my_clf

Index(['go_out', 'int_corr', 'pid', 'shar1_1', 'iid', 'attr2_1', 'sinc3_1',
       'tuition', 'imprace', 'fun3_1'],
      dtype='object')

## Задание 5

In [40]:
forest = RandomForestClassifier()
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt', 'log2', None]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}
grid = RandomizedSearchCV(estimator=forest, 
                          param_distributions=random_grid, 
                          n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
grid.fit(X, y)
grid.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.1min finished


{'n_estimators': 200,
 'min_samples_split': 2,
 'max_features': 'log2',
 'max_depth': 60}

In [41]:
arg = np.array(grid.cv_results_['mean_test_score']).argsort()
best5 = np.array(grid.cv_results_['params'])[arg][:5]
best5

array([{'n_estimators': 200, 'min_samples_split': 5, 'max_features': None, 'max_depth': 20},
       {'n_estimators': 800, 'min_samples_split': 10, 'max_features': None, 'max_depth': 30},
       {'n_estimators': 1200, 'min_samples_split': 10, 'max_features': None, 'max_depth': 10},
       {'n_estimators': 1600, 'min_samples_split': 10, 'max_features': None, 'max_depth': 20},
       {'n_estimators': 2000, 'min_samples_split': 5, 'max_features': None, 'max_depth': 50}],
      dtype=object)

Возьмём параметры из пяти лучших наборов и из дефолтного набора и сделаем GridSearch, чтобы найти победителя

In [43]:
forest = RandomForestClassifier()
n_estimators = [10, 200, 800, 1200, 1600, 2000]
max_features = ['auto', None]
max_depth = [10, 20, 30, 50, None]
min_samples_split = [2, 5, 10]
param_grid = {
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators
}
grid = GridSearchCV(estimator=forest, param_grid=param_grid, 
                          cv=3, n_jobs=-1, verbose=2)
grid.fit(X, y)
grid.best_params_

Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  6.5min finished


{'max_depth': 30,
 'max_features': 'auto',
 'min_samples_split': 10,
 'n_estimators': 10}